In [1]:
!pip install requests
!pip install pandas
!pip install numpy
!pip install seaborn 
!pip install zipfile

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [2]:
import requests
import pandas as pd
import seaborn as sns 
import numpy as np
import zipfile as zp

In [4]:
#DO NOT RUN - FOR STORAGE PURPOSES 

# Path to the original zip file
original_zip_path = 'data/NADAC-2024.csv.zip'

# Extract the CSV file from the zip archive and load it into a DataFrame
with zp.ZipFile(original_zip_path, 'r') as zipf:
    with zipf.open('NADAC-2024.csv') as file:
        df_nadac = pd.read_csv(file)

# Split the DataFrame into two halves
df_nadac_part1 = df_nadac.iloc[:len(df_nadac) // 2]
df_nadac_part2 = df_nadac.iloc[len(df_nadac) // 2:]

# Save each part to a separate zip file
with zp.ZipFile('data/NADAC-2024_part1.csv.zip', 'w') as zipf:
    zipf.writestr('NADAC-2024_part1.csv', df_nadac_part1.to_csv(index=False))

with zp.ZipFile('data/NADAC-2024_part2.csv.zip', 'w') as zipf:
    zipf.writestr('NADAC-2024_part2.csv', df_nadac_part2.to_csv(index=False))

print("Split and saved NADAC-2024.csv into two zip files: NADAC-2024_part1.csv.zip and NADAC-2024_part2.csv.zip")


Split and saved NADAC-2024.csv into two zip files: NADAC-2024_part1.csv.zip and NADAC-2024_part2.csv.zip


READING IN DATASETS 

In [3]:
import pandas as pd
import zipfile as zp

# List of zip files, including the split NADAC files
zip_files = [
    'data/CMSspending.csv.zip',
    'data/fda_directory.xlsx.zip',
    'data/NADAC-2024_part1.csv.zip',
    'data/NADAC-2024_part2.csv.zip',
]

# Create an empty dictionary to store each DataFrame
dataframes = {}

# Temporary list to hold NADAC parts
nadac_parts = []

# Loop through each zip file and extract the content
for zip_path in zip_files:
    with zp.ZipFile(zip_path, 'r') as zipf:
        # Get the name of the first file inside the zip (assuming only one file per zip)
        file_name = zipf.namelist()[0]
        
        # Extract the file and load it into a DataFrame based on its file extension
        with zipf.open(file_name) as file:
            if file_name.endswith('.csv'):
                # Load CSV file into DataFrame
                df = pd.read_csv(file)
                # If the file is a part of NADAC, add it to the nadac_parts list
                if 'NADAC-2024_part' in file_name:
                    nadac_parts.append(df)
                else:
                    # Store other CSV files normally
                    base_name = file_name.split('.')[0]
                    dataframes[base_name] = df
            elif file_name.endswith('.xlsx'):
                # Load Excel file into DataFrame
                df = pd.read_excel(file, engine='openpyxl')
                # Store the Excel file normally
                base_name = file_name.split('.')[0]
                dataframes[base_name] = df
            else:
                raise ValueError(f"Unsupported file format: {file_name}")

# Concatenate the NADAC parts into a single DataFrame and add it to the dictionary
if nadac_parts:
    dataframes['NADAC-2024'] = pd.concat(nadac_parts, ignore_index=True)
    print("Merged NADAC parts into a single DataFrame")

# Display the first few rows of each DataFrame for verification
for name, df in dataframes.items():
    print(f"\nDataFrame '{name}':")
    print(df.head())


Merged NADAC parts into a single DataFrame

DataFrame 'CMSspending':
  Utilization Type State      NDC  Labeler Code  Product Code  Package Size  \
0             MCOU    XX  2010102             2           101             2   
1             FFSU    XX  2021301             2           213             1   
2             MCOU    XX  2021301             2           213             1   
3             MCOU    XX  2035102             2           351             2   
4             FFSU    XX  2091155             2           911            55   

   Year  Quarter  Suppression Used Product Name  Units Reimbursed  \
0  2024        2             False   AMMONIUM C            21.000   
1  2024        2             False   HUMULIN R           4106.000   
2  2024        2             False   HUMULIN R           2465.532   
3  2024        2              True   DARVOCET-N               NaN   
4  2024        2              True                            NaN   

   Number of Prescriptions  Total Amount 

In [4]:
#Create dataframes for each dataset 

df_directory = dataframes["fda_directory"]
df_spending = dataframes["CMSspending"]
df_pricing = dataframes["NADAC-2024"]
df_purplebook = pd.read_csv("data/purplebook-october-data.csv", header=3)

CLEANING AND MERGING DATA

In [ ]:
#Cleaning purplebook and merging with FDA directory

clean_purple = df_purplebook[df_purplebook['Ref. Product Proprietary Name'].notna()]
clean_purple['Num Biosimilars'] = 1
grouped_purple = clean_purple.groupby('Ref. Product Proprietary Name')['Num Biosimilars'].sum().reset_index()

grouped_purple['Ref. Product Proprietary Name'] = (
    grouped_purple['Ref. Product Proprietary Name']
    .str.lower()                 # Convert to lowercase
    .str.replace(' ', '')        # Remove spaces
)


df_directory['PROPRIETARYNAME'] = (
    df_directory['PROPRIETARYNAME']
    .str.lower()                 # Convert to lowercase
    .str.replace(' ', '')        # Remove spaces
)

directory_purplebook = pd.merge(df_directory, grouped_purple, left_on='PROPRIETARYNAME', right_on="Ref. Product Proprietary Name", how='left')
directory_purplebook

In [ ]:
df = pd.read_csv('2000_sample_merge.csv', index_col=0)
temp = df[df['Suppression Used'] != True]
temp

In [152]:
import datetime

temp.columns
temp = temp[['NDC', 'NADAC Per Unit', 'Effective Date', 'As of Date', 'OTC',
       'Corresponding Generic Drug Effective Date', 'PROPRIETARYNAME',
       'DOSAGEFORMNAME', 'ROUTENAME', 'STARTMARKETINGDATE', 'ENDMARKETINGDATE',
       'MARKETINGCATEGORYNAME', 'LABELERNAME', 'SUBSTANCENAME', 'ACTIVE_NUMERATOR_STRENGTH',
       'PHARM_CLASSES', 'DEASCHEDULE', 'Num Biosimilars', 'Package Size', 'Number of Prescriptions',
       'Total Amount Reimbursed']]

y = temp[['NADAC Per Unit']]
X_temp = temp[['NDC', 'Effective Date', 'OTC', 'Corresponding Generic Drug Effective Date', 
            'STARTMARKETINGDATE',  'MARKETINGCATEGORYNAME', 'DEASCHEDULE', 'Num Biosimilars', 
            'Package Size', 'Number of Prescriptions', 'Total Amount Reimbursed']]

X_temp['Corresponding Generic Drug Effective Date'] = np.where(X_temp['Corresponding Generic Drug Effective Date'].notna(), 1, 0)
X_temp['STARTMARKETINGDATE'] = pd.to_datetime(X_temp['STARTMARKETINGDATE'], format = "ISO8601")
X_temp['Effective Date'] = pd.to_datetime(X_temp['Effective Date'], format = "mixed")
X_temp['Time in Market'] = X_temp["Effective Date"] - X_temp["STARTMARKETINGDATE"]
X_temp['Time in Market'] = X_temp['Time in Market'].dt.days.astype(int)

/var/folders/_p/0dx0pg8j0bl5scgksbwv_rbw0000gn/T/ipykernel_24688/4166595997.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_temp['Corresponding Generic Drug Effective Date'] = np.where(X_temp['Corresponding Generic Drug Effective Date'].notna(), 1, 0)
/var/folders/_p/0dx0pg8j0bl5scgksbwv_rbw0000gn/T/ipykernel_24688/4166595997.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_temp['STARTMARKETINGDATE'] = pd.to_datetime(X_temp['STARTMARKETINGDATE'], format = "ISO8601")
/var/folders/_p/0dx0pg8j0bl5

In [153]:
X_temp['Num Biosimilars'] = np.where(X_temp['Num Biosimilars'].notna(), X_temp['Num Biosimilars'], 0)
X_temp['OTC'] = np.where(X_temp['OTC'] == 'Y', 1, 0)

/var/folders/_p/0dx0pg8j0bl5scgksbwv_rbw0000gn/T/ipykernel_24688/4146921169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_temp['Num Biosimilars'] = np.where(X_temp['Num Biosimilars'].notna(), X_temp['Num Biosimilars'], 0)
/var/folders/_p/0dx0pg8j0bl5scgksbwv_rbw0000gn/T/ipykernel_24688/4146921169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_temp['OTC'] = np.where(X_temp['OTC'] == 'Y', 1, 0)


In [154]:
X = pd.get_dummies(X_temp, columns=['MARKETINGCATEGORYNAME', 'DEASCHEDULE'])

In [155]:
X = X[['NDC', 'Effective Date', 'OTC', 'Corresponding Generic Drug Effective Date', 'Num Biosimilars', 'Package Size',
       'Number of Prescriptions', 'Total Amount Reimbursed', 'Time in Market']]

In [146]:
X['year'] = X['Effective Date'].dt.year
X['month'] = X['Effective Date'].dt.month
X.columns

Index(['NDC', 'Effective Date', 'OTC',
       'Corresponding Generic Drug Effective Date', 'Num Biosimilars',
       'Package Size', 'Number of Prescriptions', 'Total Amount Reimbursed',
       'Time in Market', 'MARKETINGCATEGORYNAME_ANDA',
       'MARKETINGCATEGORYNAME_BLA', 'MARKETINGCATEGORYNAME_NDA',
       'MARKETINGCATEGORYNAME_NDA AUTHORIZED GENERIC',
       'MARKETINGCATEGORYNAME_OTC MONOGRAPH DRUG',
       'MARKETINGCATEGORYNAME_OTC MONOGRAPH FINAL',
       'MARKETINGCATEGORYNAME_OTC MONOGRAPH NOT FINAL',
       'MARKETINGCATEGORYNAME_UNAPPROVED DRUG OTHER', 'DEASCHEDULE_CII',
       'DEASCHEDULE_CIII', 'DEASCHEDULE_CIV', 'DEASCHEDULE_CV', 'year',
       'month'],
      dtype='object')

In [157]:
X.drop('Effective Date', axis=1, inplace=True)
X.drop('NDC', axis=1, inplace=True)

In [158]:
X.columns

Index(['OTC', 'Corresponding Generic Drug Effective Date', 'Num Biosimilars',
       'Package Size', 'Number of Prescriptions', 'Total Amount Reimbursed',
       'Time in Market'],
      dtype='object')

In [159]:
#Run regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Example: Splitting data into training and test sets (optional but recommended)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Instantiate the model
model = LinearRegression()

# Step 3: Fit the model
model.fit(X_train, y_train)

# Step 4: Make predictions (optional)
y_pred = model.predict(X_test)

# Evaluate the model (optional)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 4376.983207091108
R-squared: 0.022675527999384548


In [ ]:
#Time effects regression

!pip install linearmodels
import linearmodels

In [150]:
import pandas as pd
from linearmodels.panel import PanelOLS

# Assume df is your dataframe with 'NDC', 'year', and 'month' columns
# Set 'NDC' as the entity index along with 'year' and 'month' as time indices
X['y'] = y
X = X.set_index(['year', 'month'])

# Specify the fixed-effects model with entity and time fixed effects
# Replace 'y' with the dependent variable name
model = PanelOLS.from_formula(
    'y ~ OTC + Corresponding_Generic_Drug_Effective_Date + Num_Biosimilars + Package_Size + \
    Number_of_Prescriptions + Total_Amount_Reimbursed + Time_in_Market + \
    MARKETINGCATEGORYNAME_ANDA + MARKETINGCATEGORYNAME_BLA + MARKETINGCATEGORYNAME_NDA + \
    MARKETINGCATEGORYNAME_NDA_AUTHORIZED_GENERIC + MARKETINGCATEGORYNAME_OTC_MONOGRAPH_DRUG + \
    MARKETINGCATEGORYNAME_OTC_MONOGRAPH_FINAL + MARKETINGCATEGORYNAME_OTC_MONOGRAPH_NOT_FINAL + \
    MARKETINGCATEGORYNAME_UNAPPROVED_DRUG_OTHER + DEASCHEDULE_CII + DEASCHEDULE_CIII + \
    DEASCHEDULE_CIV + DEASCHEDULE_CV + EntityEffects + TimeEffects',
    data=df
)

# Fit the model
fe_results = model.fit()

# Display the summary
print(fe_results.summary)


ValueError: Length of values (16015) does not match length of index (947427)

EXTRANEOUS

In [3]:
url = "https://data.medicaid.gov/api/1/datastore/query/99315a95-37ac-4eee-946a-3c523b4c481e/0"

In [ ]:
database = []
offset = 0
while True:
    response = requests.get(url, params={'limit': 5000, 'offest': offset, 'format': 'json'})
    data = response.json().get('results', [])
    if not data:
        break
    print('iteration')
    database.extend(data)
    offset += 5000

medicaid_df = pd.DataFrame(database)